In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, fftpack
import time
import datetime
# from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

import fnmatch
import os
import shutil
import sys
import csv

# suppress matplotlib deprecation warnings
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

%matplotlib inline

# to clear outputs from cells
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [ ]:
def get_min_max(x):

    # flatten the input array http://bit.ly/2MQuXZd
    flat_vector = np.concatenate(x).ravel()

    min_val = min(flat_vector)
    max_val = max(flat_vector)

    return min_val, max_val


def scaler(x, min_val, max_val, lower_norm_val=0, upper_norm_val=1):
    """Scale the signal between a min and max value
    
    Parameters
    ===========
    x : ndarray
        Signal that is being normalized

    max_val : int or float
        Maximum value of the signal or dataset

    min_val : int or float
        Minimum value of the signal or dataset

    lower_norm_val : int or float
        Lower value you want to normalize the data between (e.g. 0)

    upper_norm_val : int or float
        Upper value you want to normalize the data between (e.g. 1)

    Returns
    ===========
    x : ndarray
        Returns a new array that was been scaled between the upper_norm_val
        and lower_norm_val values

    """

    # https://codereview.stackexchange.com/questions/185785/scale-numpy-array-to-certain-range
    col, row = np.shape(x)
    for i in range(col):
        x[i] = np.interp(x[i], (min_val, max_val), (lower_norm_val, upper_norm_val))
    return x

In [ ]:
# set the root (parent folder) and the data folder locations
folder_root = Path.cwd().parent # get root folder of repository

folder_raw_data = folder_root / 'data/raw/FEMTO/Training_set/Learning_set/' # raw data folder

In [ ]:
# load text file for first measurement
# first test folder location
folder_bearing1_1 = folder_raw_data / 'Bearing1_1'

# let's use pandas
col_names = ['hr', 'min', 'sec', 'micro_sec', 'acc_horz', 'acc_vert']
df = pd.read_csv(folder_bearing1_1 / 'acc_00001.csv', names=col_names)

In [ ]:
df.head()

In [ ]:
# what is the shape of the dataframe?
df.shape

From the IEEE PHM 2012 Prognostic challenge outline, it says that the collection is at 25.6 kHz for the acceleration signals

Other important operating information:

* First operating conditions: 1800 rpm and 4000 N
* Second operating condition: 1650 rpm and 4200 N
* Third operating condition: 1500 rpm and 5000 N

In [ ]:
# plot first bearing channel
fig, ax = plt.subplots()

ax.plot(
    np.arange(0,df.shape[0], dtype='float64') / (25.6 * 10**3), # make x-axis in seconds
    df['acc_horz'] # acceleration data
)

In [ ]:
# practice detrending
fig, ax = plt.subplots(1, 1, figsize=(15, 8))

plt.plot(df['acc_horz'], alpha=0.5, label='original signal')
y_detrend = signal.detrend(df['acc_horz'], type="linear")
plt.plot(y_detrend, alpha=0.5, label='detrended signal')

# apply either a hamming or kaiser windowing function
# y_detrend *= np.hamming(len(y_detrend))
y_detrend *= np.kaiser(len(y_detrend), 3)
plt.plot(y_detrend, alpha=0.5, label='windowed signal')
plt.legend(loc='center left')

Create a generic function for plotting the FFT.

In [ ]:
def create_fft(df, x_name='Time', y_name='acc_horz', sample_freq=25600.0, show_plot=True, window='hamming', beta=8):
    '''Create FFT plot from a pandas dataframe'''

    y = df[y_name].to_numpy(dtype="float64")  # convert to a numpy array
    x = np.arange(0,df.shape[0], dtype='float64') / (sample_freq)

    # parameters for plot
    T = 1.0 / sample_freq  # sample spacing
    N = len(y)  # number of sample points
    
    # do some preprocessing of the current signal
    y_detrend = y - np.mean(y)
    y_detrend = signal.detrend(y_detrend, type="constant")  # detrended signal
    
    if window == 'hamming':
        y_detrend *= np.hamming(N)  # apply a hamming window. Why? https://dsp.stackexchange.com/a/11323
    else:
        y_detrend *= np.kaiser(len(y_detrend), beta)

    # FFT on time domain signal
    yf = fftpack.rfft(y_detrend)
    yf = 2.0 / N * np.abs(yf[: int(N / 2.0)])
    xf = np.linspace(0.0, 1.0 / (2.0 * T), N // 2)/2

    if show_plot:
        # setup the seaborn plot
        sns.set(font_scale=1.0, style="whitegrid")
        fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False, sharey=False)
        fig.tight_layout(pad=5.0)

        pal = sns.cubehelix_palette(6, rot=-0.25, light=0.7)  # pick nice color for plot

        # plot time domain signal
        axes[0].plot(x, y, marker="", label="Best model", color=pal[3], linewidth=0.8)
        axes[0].set_title("Time Domain", fontdict={"fontweight": "normal"})
        axes[0].set_xlabel("Time (seconds)")
        axes[0].set_ylabel("Acceleration, g")
        # axes[0].set_yticklabels([])

        # plot the frequency domain signal
        axes[1].plot(xf, yf, marker="", label="Best model", color=pal[3], linewidth=0.8)
        axes[1].set_title("Frequency Domain", fontdict={"fontweight": "normal"})
        axes[1].set_xlabel("Frequency (Hz)")
        axes[1].set_ylabel("Acceleration, g")
        plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
        

        # clean up the sub-plots to make everything pretty
        for ax in axes.flatten():
            ax.yaxis.set_tick_params(labelleft=True, which="major")
            ax.grid(False)
            
        # in case you want to save the figure (just uncomment)
        plt.savefig('time_freq_domains.png',dpi=600,bbox_inches = "tight")
        plt.show()
    
    return xf, yf

xf, yf = create_fft(df, x_name='Time', y_name='acc_horz', sample_freq=25600.0, show_plot=True, window='kaiser', beta=3)

In [ ]:
def plot_freq(xf, yf, max_freq_to_plot=1000, peak_height=0.0001, peak_distance=100):

    # select the index number where xf is less than a certain freq
    i = np.where(xf<max_freq_to_plot)[0][-1]
    peak_distance_index = peak_distance * i / max_freq_to_plot

    # setup the seaborn plot
    sns.set(font_scale=1.0, style="whitegrid")
    fig, axes = plt.subplots(1, 1, figsize=(12, 8), sharex=False, sharey=False)
    fig.tight_layout(pad=5.0)
    
    pal = sns.cubehelix_palette(6, rot=-0.25, light=0.7)  # pick nice color for plot

    # plot the frequency domain signal
    axes.plot(xf[:i], yf[:i], marker="", label="Best model", color=pal[3], linewidth=0.8)
    axes.set_title("Frequency Domain", fontdict={"fontweight": "normal"})
    axes.set_xlabel("Frequency (Hz)")
    axes.set_ylabel("Acceleration, g")
    axes.yaxis.set_tick_params(labelleft=True, which="major")
    axes.grid(False)

    peaks, _ = signal.find_peaks(yf[:i], height=peak_height, distance=peak_distance_index)
    plt.plot(xf[peaks], yf[peaks], "x", color='#d62728', markersize=10)

    for p in peaks:
        axes.text(
            x=xf[p]+max_freq_to_plot/50.0,
            y=yf[p],
            s=f"{xf[p]:.1f} Hz",
            horizontalalignment="left",
            verticalalignment="center",
            size=12,
            color="#d62728",
            rotation="horizontal",
            weight="normal",
        )

    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.show()


plot_freq(xf, yf, max_freq_to_plot=6000, peak_height=0.06, peak_distance=250)

Now we'll try a different operating condition -- how about the third operating condition at 1500 RPM?

In [ ]:
# third operating condition folder location
folder_bearing3_1 = folder_raw_data / 'Bearing3_1'

# let's use pandas
col_names = ['hr', 'min', 'sec', 'micro_sec', 'acc_horz', 'acc_vert']
df = pd.read_csv(folder_bearing3_1 / 'acc_00001.csv', names=col_names)

In [ ]:
xf, yf = create_fft(df, x_name='Time', y_name='acc_horz', sample_freq=25600.0, show_plot=True, window='kaiser', beta=3)

In [ ]:
# compare to bearing1_1
df = pd.read_csv(folder_bearing1_1 / 'acc_00001.csv', names=col_names)
xf, yf = create_fft(df, x_name='Time', y_name='acc_horz', sample_freq=25600.0, show_plot=True, window='kaiser', beta=3)

In [ ]:
len(xf)

## Spectrogram
We want to build a spectrogram for bearing1_1.

First, get the name of all the files in the folder.

### Aside: Getting Acceleration Files Only

In [ ]:
def create_date_dict(folder):
    # instantiate the date dictionary that will
    # hold the date/time that each signal was recorded on
    # along with the file name

    date_dict = {}

    for i, file in enumerate(os.listdir(folder)):
        if fnmatch.fnmatch(file, f'acc*.csv'):

            # get the unix timestamp for when the file was modified (http://bit.ly/2RW5cYo)
            date_created = datetime.datetime.fromtimestamp(os.path.getmtime(folder_bearing1_1 / str(file)))

            # open each csv file, read first line, and extract times
            with open(folder_bearing1_1 / file, newline='') as f:
                csv_reader = csv.reader(f)
                csv_headings = next(csv_reader)

            # help with datetime: https://realpython.com/python-datetime/
            # convert "time" string to datetime object
            time_created = datetime.time(hour=int(float(csv_headings[0])), 
                                         minute=int(float(csv_headings[1])), 
                                         second=int(float(csv_headings[2])), 
                                         microsecond=int(float(csv_headings[3]))
                                         )

            # combine date and time into a single datetime object
            combined_date = datetime.datetime.combine(date_created, time_created)
            unix_timestamp = combined_date.timestamp()
            date_nice_format = combined_date.strftime('%Y-%m-%d %H:%M:%S')

            date_dict[unix_timestamp] = [combined_date, date_nice_format, file]
    return date_dict


In [ ]:
folder_bearing1_1 = folder_raw_data / 'Bearing1_1'

date_dict = create_date_dict(folder_bearing1_1)

In [ ]:
date_dict

### Calculate FFT for Each Signal
We'll calculate the FFT for each signal. This will be stored in a pandas dataframe, with each new column being a new signal.

In [ ]:
def build_spectrogram_df_femto(folder, date_dict, channel_name='acc_horz', col_day_increment=False,
                         col_names=['hr', 'min', 'sec', 'micro_sec', 'acc_horz', 'acc_vert']):
    '''function that builds the spectrogram data'''
    
    # date_time list
    date_list = sorted(list(date_dict.keys()))
    start_time = date_list[0] # get the star time

    # instantiate dataframe for the spectrogram
    dft = pd.DataFrame()
       
    # dictionary to store any labels
    labels_dict = {}

    # iterate through each date that samples were taken
    # date_list should be sorted from earliest to latest
    for i, unix_timestamp in enumerate(date_list):
        # convert sample_name to unix timestamp
        date_nice_format = date_dict[unix_timestamp][1]

        # open the file containing the measurements
        df = pd.read_csv(folder / date_dict[unix_timestamp][2], names=col_names)

        # create fft
        xf, yf = create_fft(df, x_name='Time', y_name=channel_name, sample_freq=25600.0, show_plot=False, window='kaiser', beta=3)
        # xf, yf = create_fft(df, x_name='Time', y_name=channel_name, sample_freq=20000.0, show_plot=False, window='kaiser', beta=3)


        # append the time increments
        time_increment_seconds = unix_timestamp-start_time
        time_increment_days = time_increment_seconds /(60 * 60 * 24)
        
        # create new column for the current sample_name FFT
        if col_day_increment == False:
            dft[date_nice_format] = yf
        if col_day_increment == True:
            dft[str(time_increment_days)] = yf

        # create new dictionary key and values to store lable info
        labels_dict[unix_timestamp] = [date_nice_format, unix_timestamp, time_increment_seconds, time_increment_days]

    dft = dft.set_index(xf, drop=True) # index as frequency (Hz)
    return dft, labels_dict

In [ ]:
folder_bearing1_1 = folder_raw_data / 'Bearing1_1'

date_dict = create_date_dict(folder_bearing1_1)

df_spec, labels_dict = build_spectrogram_df_femto(folder_bearing1_1, date_dict, channel_name='acc_horz',)
df_spec.head()

Plot the spectrogram.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.pcolormesh(df_spec.columns, df_spec.index, df_spec)

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::200]) # show every 100th date on x-axis ticks
plt.xticks(rotation=75)

plt.show()

The above spectrogram is fairly "dim". 

We'll adjust the vmax to get better clarity.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# set vmax to 0.2
plt.pcolormesh(df_spec.columns, df_spec.index, df_spec, vmax=0.2)

ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_spec.columns[::100]) # show every 100th date on x-axis ticks
plt.xticks(rotation=75)

plt.show()

## Build Features

In [ ]:
folder_bearing1_1 = folder_raw_data / 'Bearing1_1'

date_dict = create_date_dict(folder_bearing1_1)

df_spec, labels_dict = build_spectrogram_df_femto(folder_bearing1_1, date_dict, channel_name='acc_horz',)

In [ ]:
df_spec.shape

In [ ]:
bucket_size = 64

df_temp = df_spec
a = np.array(df_temp) # make numpy array

# get the y-axis (frequency values)
y = np.array(df_temp.index)
y = np.max(y.reshape(-1,bucket_size),axis=1)

# get the max value for each bucket
# https://stackoverflow.com/a/15956341/9214620
max_a = np.max(a.reshape(-1,bucket_size,2802),axis=1)

print('shape of max_a array:', np.shape(max_a))

# get the mean value for each bucket
avg_a = np.mean(a.reshape(-1,bucket_size,2802),axis=1)

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
plt.pcolormesh(df_temp.columns, y, max_a)
ax.set_ylabel("Frequency (Hz)")
plt.xticks(df_temp.columns[::200])
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.plot(max_a.T)
plt.show()